In [62]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import random
import csv

# --- KONFIGURASI ---
user_ig = "masteedarkflame"
pass_ig = "Aviator99"
hashtag = "menterikeuangan"
TARGET_POSTS = 50
COMMENTS_PER_POST = 200

chrome_options = Options()
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 20)

def smart_click_not_now():
    """Menangani pop-up yang sering muncul setelah login"""
    buttons = [
        "//div[@role='button' and contains(text(), 'Not now')]",
        "//button[contains(text(), 'Not Now')]",
    ]
    for xpath in buttons:
        try:
            element = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            element.click()
            time.sleep(2)
        except:
            continue

def refresh_coordinates():
    print("Sedang membersihkan koordinat scroll...")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)
    for i in range(1, 5):
        pixels = i * 800
        driver.execute_script(f"window.scrollTo(0, {pixels});")
        time.sleep(random.uniform(1.0, 2.0))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

In [63]:
try:
    # 1. LOGIN
    driver.get("https://www.instagram.com")
    wait.until(EC.presence_of_element_located((By.NAME, "username"))).send_keys(user_ig)
    wait.until(EC.presence_of_element_located((By.NAME, "password"))).send_keys(pass_ig)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

    print("Login berhasil, melewati pop-up...")
    time.sleep(8)
    smart_click_not_now()

    # 2. MENGUMPULKAN LINK POSTINGAN
    driver.get(f"https://www.instagram.com/explore/tags/{hashtag}/")
    post_links = set()
    print(f"Mencari {TARGET_POSTS} link postingan...")

    last_count = 0
    stuck_counter = 0

    while len(post_links) < TARGET_POSTS:
        elements = driver.find_elements(
            By.XPATH,
            "//a[contains(@href, '/p/') or contains(@href, '/reels/')]"
        )

        for el in elements:
            link = el.get_attribute("href")
            post_links.add(link)
            if len(post_links) >= TARGET_POSTS:
                break

        if len(post_links) == last_count:
            stuck_counter += 1
        else:
            last_count = len(post_links)
            stuck_counter = 0

        if stuck_counter >= 3:
            refresh_coordinates()
            stuck_counter = 0
        else:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(3, 5))

        print(f"Total link didapat: {len(post_links)}")

    # 3. SCRAPING KOMENTAR DENGAN INTERNAL SCROLL
    with open('hasil_scraping_ig.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Link Post', 'Komentar'])

        for i, link in enumerate(list(post_links)):
            print(f"\n[{i+1}/{TARGET_POSTS}] Memproses: {link}")
            driver.get(link)
            time.sleep(random.uniform(5, 7))

            comments_found = set()
            last_height = 0

            xpath_scroll_internal = "/html/body/div[1]/div/div/div[2]/div/div/div[1]/div[1]/div[1]/section/main/div/div[1]/div/div[2]/div/div[2]"
            commentar = "//span[contains(@class, 'x1lliihq') and contains(@class, 'x1plvlek')]"

            try:
                comment_section = wait.until(
                    EC.presence_of_element_located((By.XPATH, xpath_scroll_internal))
                )

                for scroll_attempt in range(25):
                    spans = driver.find_elements(By.XPATH, commentar)

                    for s in spans:
                        text = s.text.strip()
                        if text and " " in text and len(text) > 5:
                            comments_found.add(text)

                    if len(comments_found) >= COMMENTS_PER_POST:
                        break

                    driver.execute_script(
                        "arguments[0].scrollTop = arguments[0].scrollHeight",
                        comment_section
                    )
                    time.sleep(random.uniform(2, 3))

                    new_height = driver.execute_script(
                        "return arguments[0].scrollHeight",
                        comment_section
                    )

                    if new_height == last_height:
                        try:
                            load_more = driver.find_element(
                                By.XPATH,
                                "//*[local-name()='svg' and (@aria-label='Load more comments' or @aria-label='Muat komentar lainnya')]/.."
                            )
                            driver.execute_script("arguments[0].click();", load_more)
                            time.sleep(2)
                        except:
                            break

                    last_height = new_height

            except Exception as e:
                print(f"Melewati postingan ini karena: {e}")

            final_comments = list(comments_found)[:COMMENTS_PER_POST]
            for comment in final_comments:
                writer.writerow([link, comment])

            file.flush()
            print(f"Tersimpan {len(final_comments)} komentar.")

except Exception as e:
    print(f"Terjadi Error Global: {e}")

finally:
    print("\n--- PROSES SELESAI ---")
    print("Data akhir tersimpan di: hasil_scraping_ig.csv")
    driver.quit()


Login berhasil, melewati pop-up...
Mencari 50 link postingan...
Total link didapat: 0
Total link didapat: 19
Total link didapat: 36
Total link didapat: 45
Total link didapat: 48
Total link didapat: 50

[1/50] Memproses: https://www.instagram.com/p/DQ0E5IdE0Et/
Tersimpan 11 komentar.

[2/50] Memproses: https://www.instagram.com/p/DQVzS_CEi5W/
Tersimpan 126 komentar.

[3/50] Memproses: https://www.instagram.com/p/DQUIf03kufG/
Tersimpan 17 komentar.

[4/50] Memproses: https://www.instagram.com/p/DO7_VhWDrP_/
Tersimpan 135 komentar.

[5/50] Memproses: https://www.instagram.com/p/DQJDva2k7GT/
Tersimpan 131 komentar.

[6/50] Memproses: https://www.instagram.com/p/CL6tJ9lgOQf/
Tersimpan 200 komentar.

[7/50] Memproses: https://www.instagram.com/p/CQI2LTHtXml/
Tersimpan 194 komentar.

[8/50] Memproses: https://www.instagram.com/p/DQYxOOxk3dI/
Tersimpan 16 komentar.

[9/50] Memproses: https://www.instagram.com/p/DP0CP9REx5e/
Tersimpan 15 komentar.

[10/50] Memproses: https://www.instagram.com/p